# As due to privacy Reasons we cannot show the cookies dataset publicly hence after running the code I deleted the data set

# 🚀 How to Run ShikshaAI (Non-Interactive Version)
ShikshaAI automatically converts YouTube lectures into structured study packs — no user input required during execution. It runs end-to-end using configuration values and preloaded cookies.
# 📁 Step 1: Upload YouTube Cookies
### To bypass bot detection and age restrictions:
* Log in to YouTube in your browser.
* Use the Get cookies.txt extension.
* Export cookies while viewing a video.
* Upload the cookies.txt file to Kaggle as a dataset.
* The notebook auto-detects and copies it to /kaggle/working/cookies.txt.

# ⚙️ Step 2: Configure config.yaml
#### No prompts are used — everything is driven by config:

video_ids: "UdE-W30oOXo"  # Comma-separated YouTube video IDs

chunk_length: 600         # Max audio chunk length in seconds

output_dir: "output"      # Where Markdown files are saved

max_workers: 2            # Parallelism for multi-video processing

cookies_file: ""          # Leave blank; auto-detected by setup_cookies()

#### To change the yt video change the video_ids section and aslo change the cookies file for new video

# ▶️ Step 3: Run the Notebook
Execute all cells top-to-bottom.
The pipeline will:
* Download audio from YouTube.
* Transcribe using Whisper.
* Summarize using fallback APIs (APIfy, Groq, Mistral).
* Generate flashcards and quizzes.
* Export everything to output/ShikshaAI_Output_<video_id>.md.

# 📦 Output
* 📺 Video URL
* 📝 Summary
* 🎯 Flashcards
* 🧪 Quiz
* 🎤 Transcript

# 1. Setup & Dependencies
### Purpose: - To Install required libraries and set up the environment. ###


In [1]:
!pip install -q openai-whisper cohere mistralai yt-dlp pyyaml requests ffmpeg-python --upgrade
!apt-get -y update && apt-get -y install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s e

### Explanation: ###

* Installs Python packages for audio processing, API clients, and YouTube downloads.
* Installs ffmpeg for audio/video processing.

# 2. Imports & Logging #
### Purpose: Import libraries and configure logging. ###


In [2]:
import os
import re
import yaml
import logging
import warnings
import subprocess
import whisper
import ffmpeg
import requests
import sys
from typing import Optional
from mistralai import Mistral
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from kaggle_secrets import UserSecretsClient



# -------------------------
# Logging & warnings
# -------------------------
warnings.filterwarnings("ignore", category=UserWarning)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("shikshaai")


### Explanation: ###
* Imports all necessary libraries.
* Configures logging to track script execution.

# 3. Configuration #
### Purpose: Load and manage configuration settings. ###


In [3]:
# -------------------------
# Config
# -------------------------
CONFIG_PATH = "config.yaml"
DEFAULT_CONFIG = """
output_dir: "./output"
max_workers: 1
chunk_length: 600 # seconds (10 minutes)
cookies_file: "" # Path to Netscape-format cookies.txt for YouTube auth (optional)
"""
if not os.path.exists(CONFIG_PATH):
    with open(CONFIG_PATH, "w") as f:
        f.write(DEFAULT_CONFIG)
def load_config():
    with open(CONFIG_PATH, "r") as f:
        return yaml.safe_load(f)
config = load_config()
os.makedirs(config["output_dir"], exist_ok=True)

### Explanation: ###
* Creates a default config.yaml if it doesn’t exist.
* Loads configuration settings for output directory, workers, and chunk length.

# 4. API Keys & Clients
### Purpose: Load API keys and initialize clients.


In [4]:
# -------------------------
# Kaggle Secrets
# -------------------------
try:
    user_secrets = UserSecretsClient()
    APIFY_KEY = user_secrets.get_secret("APIFY_API_KEY")
    GROQ_KEY = user_secrets.get_secret("GROQ_API_KEY")
    MISTRAL_KEY = user_secrets.get_secret("MISTRAL_API_KEY")
except Exception as e:
    # Fallback for local testing if not on Kaggle
    logger.warning("Could not load Kaggle secrets. Ensure you are on Kaggle or set env vars manually.")
    APIFY_KEY = os.getenv("APIFY_API_KEY")
    GROQ_KEY = os.getenv("GROQ_API_KEY")
    MISTRAL_KEY = os.getenv("MISTRAL_API_KEY")
if not APIFY_KEY or not GROQ_KEY or not MISTRAL_KEY:
    # Don't raise error immediately, allow script to compile, but fail later if needed
    logger.warning("⚠️ One or more API keys are missing! The pipeline will fail at the API step.")
# Clients
if GROQ_KEY:
    groq_client = OpenAI(base_url="https://api.groq.com/openai/v1", api_key=GROQ_KEY)
if MISTRAL_KEY:
    mistral_client = Mistral(api_key=MISTRAL_KEY)

### Explanation:
* Loads API keys from Kaggle secrets or environment variables.
* Initializes clients for Groq and Mistral APIs.

# 5. Helper Functions
### Purpose: Utility functions for video ID extraction, filename sanitization, and audio processing.


In [5]:
# -------------------------
# Helpers
# -------------------------
def extract_video_id(url: str) -> str:
    m = re.search(r"(?:youtu\.be/|v=)([A-Za-z0-9_-]{6,})", url)
    return m.group(1) if m else "video"
def safe_filename(s: str) -> str:
    return "".join(c if c.isalnum() or c in "-_." else "_" for c in s)
def get_audio_duration(file_path: str) -> float:
    try:
        probe = ffmpeg.probe(file_path)
        return float(probe["format"]["duration"])
    except ffmpeg.Error as e:
        logger.error(f"FFmpeg probe failed: {e.stderr.decode() if e.stderr else str(e)}")
        return 0.0
def split_audio(input_file: str, chunk_length: int = 600):
    if os.path.exists("chunks"):
        import shutil
        shutil.rmtree("chunks")
    os.makedirs("chunks", exist_ok=True)
       
    cmd = [
        "ffmpeg", "-y", "-i", input_file,
        "-f", "segment",
        "-segment_time", str(chunk_length),
        "-c", "copy", "chunks/out%03d.mp3"
    ]
    subprocess.run(cmd, check=True)
def choose_whisper_model(duration: float) -> str:
    if duration < 600: # <10 min
        return "base"
    elif duration < 3600: # <1 hour
        return "small"
    else:
        return "medium"

### Explanation:
* extract_video_id: Extracts YouTube video ID from URL.
* safe_filename: Sanitizes filenames.
* get_audio_duration: Gets audio duration using ffmpeg.
* split_audio: Splits audio into chunks for processing.
* choose_whisper_model: Selects Whisper model based on audio duration.

# 6. Transcript Agent
### Purpose: Downloads audio from YouTube and transcribes it using Whisper.


In [6]:
# -------------------------
# Transcript (Local Whisper)
# -------------------------
class TranscriptAgent:
    def __init__(self, chunk_length: int):
        self.chunk_length = chunk_length

    def download_audio(self, url: str, out_path: str):
        logger.info("Downloading audio with yt-dlp...")
        cookies_file = config.get("cookies_file", "").strip()
        cmd = [
            "yt-dlp", "--no-warnings", "--cookies", cookies_file,
            "-f", "bestaudio/best", "-x", "--audio-format", "mp3",
            "-o", out_path, url
        ]

        try:
            subprocess.run(cmd, check=True)
        except subprocess.CalledProcessError:
            logger.warning("⚠️ bestaudio failed, retrying with fallback format...")
            fallback_cmd = [
                "yt-dlp", "--no-warnings", "--cookies", cookies_file,
                "-f", "best", "-x", "--audio-format", "mp3",
                "-o", out_path, url
            ]
            subprocess.run(fallback_cmd, check=True)

    def transcribe(self, url: str) -> str:
        vid = extract_video_id(url)
        audio_file = f"temp_{vid}.mp3"
        if os.path.exists(audio_file):
            os.remove(audio_file)

        self.download_audio(url, audio_file)

        if not os.path.exists(audio_file):
            raise FileNotFoundError(f"Audio file not found: {audio_file}")

        duration = get_audio_duration(audio_file)
        model_name = choose_whisper_model(duration)
        logger.info(f"Loading Whisper model: {model_name}")
        model = whisper.load_model(model_name)

        if duration <= self.chunk_length:
            result = model.transcribe(audio_file)
            return result["text"].strip()

        split_audio(audio_file, self.chunk_length)
        transcript_parts = []
        for chunk in sorted([c for c in os.listdir("chunks") if c.endswith(".mp3")]):
            result = model.transcribe(os.path.join("chunks", chunk))
            transcript_parts.append(result["text"])
        return "\n".join(transcript_parts).strip()



### Explanation:
* Downloads audio from YouTube using yt-dlp.
* Transcribes audio using Whisper, splitting long audio into chunks.

# 7. Summarization
### Purpose: Summarizes transcripts using APIfy, Groq, or Mistral.


In [7]:
# -------------------------
# Summarization
# -------------------------
def summarize_with_apify(transcript: str) -> Optional[str]:
    if not APIFY_KEY: return None
    url = "https://api.apify.com/v2/acts/easyapi/text-summarization/run-sync"
    payload = {"text": transcript, "output_sentences": 5}
    headers = {"Authorization": f"Bearer {APIFY_KEY}"}
    try:
        resp = requests.post(url, json=payload, headers=headers, timeout=120)
        if resp.status_code == 200:
            data = resp.json()
            output = data.get("output", [])
            if isinstance(output, list) and len(output) > 0:
                summary = " ".join([item.get("text", "") for item in output if isinstance(item, dict)])
                return summary.strip() if summary else None
            summary = data.get("summary") or data.get("output", {}).get("summary", "")
            return summary.strip() if summary else None
    except Exception as e:
        logger.warning(f"APIfy error: {e}")
    return None
def summarize_with_groq(transcript: str) -> Optional[str]:
    if not GROQ_KEY: return None
    try:
        resp = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": f"Summarize this lecture transcript into a clear outline:\n\n{transcript[:15000]}"}],
            temperature=0.3,
            max_tokens=1000
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        logger.warning(f"Groq error: {e}")
        return None
def summarize_with_mistral(transcript: str) -> Optional[str]:
    if not MISTRAL_KEY: return None
    try:
        resp = mistral_client.chat.complete(
            model="mistral-small-2409",
            messages=[{"role": "user", "content": f"Summarize this lecture transcript into structured notes:\n\n{transcript[:15000]}"}],
            temperature=0.3,
            max_tokens=1200
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        logger.warning(f"Mistral error: {e}")
        return None
def generate_summary(transcript: str) -> str:
    # Try providers in order
    summary = summarize_with_apify(transcript)
    if summary: return summary
   
    summary = summarize_with_groq(transcript)
    if summary: return summary
       
    summary = summarize_with_mistral(transcript)
    if summary: return summary
       
    raise RuntimeError("All summarization providers failed.")

### Explanation:
* Attempts to summarize transcripts using APIfy, Groq, or Mistral in sequence.

# 8. Flashcards & Quiz Generation
### Purpose: Generates flashcards and quizzes from summaries.


In [8]:
# -------------------------
# Flashcards & Quiz
# -------------------------
def generate_flashcards(summary: str) -> str:
    if not GROQ_KEY: return "⚠️ GROQ_KEY missing."
    try:
        resp = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content":
                       f"Generate 15 Q&A flashcards from this summary. Format: Q: ... A: ...\n\n{summary}"}],
            temperature=0.4,
            max_tokens=1200
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"
def generate_quiz(summary: str) -> str:
    if not MISTRAL_KEY: return "⚠️ MISTRAL_KEY missing."
    try:
        resp = mistral_client.chat.complete(
            model="mistral-small-2409",
            messages=[{"role": "user", "content":
                       f"Create a 10-question MCQ quiz with answers and rationales based on:\n{summary}"}],
            temperature=0.4,
            max_tokens=1500
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

### Explanation:
* Uses Groq and Mistral to generate flashcards and quizzes.

# 9. Export Agent
### Purpose: Saves output as a Markdown file.


In [9]:
# -------------------------
# Export
# -------------------------
class ExportAgent:
    def save_markdown(self, url: str, transcript: str, summary: str, flashcards: str, quiz: str):
        md = f"""# 📘 ShikshaAI Study Pack
---
## 📺 URL
{url}
---
## 📝 Summary
{summary}
---
## 🎯 Flashcards
{flashcards}
---
## 🧪 Quiz
{quiz}
---
## 🎤 Transcript (Local Whisper)
{transcript}
"""
        base_id = extract_video_id(url)
        output_file = os.path.join(config["output_dir"], f"ShikshaAI_Output_{safe_filename(base_id)}.md")
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(md)
        logger.info(f"📄 Exported: {output_file}")

### Explanation:
* Formats and saves the study pack as a Markdown file.

# 10. Pipeline & Main Function
### Purpose: Orchestrates the entire process.


In [10]:

# -------------------------
# Pipeline
# -------------------------
def process_url(url: str):
    try:
        logger.info(f"\n===== 🔄 Processing: {url} =====")
        transcriber = TranscriptAgent(config["chunk_length"])
        exporter = ExportAgent()
       
        transcript = transcriber.transcribe(url)
        summary = generate_summary(transcript)
        flashcards = generate_flashcards(summary)
        quiz = generate_quiz(summary)
       
        exporter.save_markdown(url, transcript, summary, flashcards, quiz)
        logger.info(f"🎉 DONE! Study pack for {url} is ready.")
    except Exception as e:
        logger.error(f"❌ Error in pipeline for {url}: {e}")

def input_urls() -> list:
    # Provide IDs programmatically instead of prompting
    ids_raw = config.get("video_ids", "h0e2HAPTGF4")  # fallback default
    urls = [f"https://youtu.be/{id.strip()}" for id in ids_raw.split(",") if id.strip()]
    return urls

        
# -------------------------
# Safe Cookies Handling
# -------------------------
def setup_cookies():
    """
    Safely use cookies:
    - Copy to /kaggle/working so yt-dlp can write to it
    - But delete BEFORE Kaggle generates output
    """

    for root, dirs, files in os.walk("/kaggle/input"):
        if "cookies.txt" in files:
            src = os.path.join(root, "cookies.txt")
            dst = "/kaggle/working/cookies.txt"

            import shutil
            shutil.copy(src, dst)

            # Use temporary location for yt-dlp
            config["cookies_file"] = dst
            return

    logger.warning("⚠️ No cookies.txt found — yt-dlp may fail.")





def main():
    setup_cookies()
    urls = input_urls()  # now returns a list without prompting
    try:
        with ThreadPoolExecutor(max_workers=config["max_workers"]) as executor:
            executor.map(process_url, urls)
    except KeyboardInterrupt:
        logger.info("Process interrupted by user.")
    except Exception as e:
        logger.error(f"❌ ERROR in main pipeline: {e}")


if __name__ == "__main__":
    main()

# ---------------------------------------------------
# 🧹 DELETE cookies BEFORE Kaggle saves output (SAFE)
# ---------------------------------------------------
try:
    cookies_path = config.get("cookies_file")
    if cookies_path and os.path.exists(cookies_path):
        os.remove(cookies_path)
        logger.info("Cookies removed from working directory for safety.")
except:
    pass

2025-11-30 18:22:45,435 - INFO - 
===== 🔄 Processing: https://youtu.be/h0e2HAPTGF4 =====
2025-11-30 18:22:45,436 - INFO - Downloading audio with yt-dlp...


[youtube] Extracting URL: https://youtu.be/h0e2HAPTGF4
[youtube] h0e2HAPTGF4: Downloading webpage
[youtube] h0e2HAPTGF4: Downloading tv downgraded player API JSON
[youtube] h0e2HAPTGF4: Downloading web safari player API JSON
[youtube] h0e2HAPTGF4: Downloading m3u8 information
[info] h0e2HAPTGF4: Downloading 1 format(s): 96
[download] Sleeping 6.00 seconds as required by the site...
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 553
[download] Destination: temp_h0e2HAPTGF4.mp4
[download] 100% of  379.21MiB in 00:00:43 at 8.77MiB/s                   
[FixupM3u8] Fixing MPEG-TS in MP4 container of "temp_h0e2HAPTGF4.mp4"
[ExtractAudio] Destination: temp_h0e2HAPTGF4.mp3
Deleting original file temp_h0e2HAPTGF4.mp4 (pass -k to keep)


2025-11-30 18:24:20,561 - INFO - Loading Whisper model: small
100%|████████████████████████████████████████| 461M/461M [00:01<00:00, 287MiB/s]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enabl

### Explanation:

* process_url: Processes a single URL.
* input_urls: Returns a list of YouTube URLs.
* setup_cookies: Copies cookies for YouTube authentication.
* main: Orchestrates the entire pipeline.
